In [8]:
import pandas as pd

# Read all CSVs and rename columns using the column names
# Heads up, left side of colon is my CSV names I downloaded you will have to change to match what your file names are
mdel = pd.read_csv('MDEL.csv').rename(columns={'DRSFRMACBS': 'delinquency_rate'})
m30yr = pd.read_csv('M30YR.csv').rename(columns={'MORTGAGE30US': '30yr_fixed_rate'})
mfico = pd.read_csv('MFICO.csv').rename(columns={'OBMMIC30YFLVLE80FGE740': '30yr_conforming_fico740'})
mortgage15us = pd.read_csv('MORTGAGE15US.csv').rename(columns={'MORTGAGE15US': '15yr_fixed_rate'})
mbs = pd.read_csv('MBS.csv').rename(columns={'TMBACBW027SBOG': 'mbs_price'})

# Merge all dataframes outer merge to keep all possible data
merged_df = mdel.merge(m30yr, on='observation_date', how='outer') \
                .merge(mfico, on='observation_date', how='outer') \
                .merge(mortgage15us, on='observation_date', how='outer') \
                .merge(mbs, on='observation_date', how='outer')

# Sort by the dates
merged_df = merged_df.sort_values('observation_date').reset_index(drop=True)
merged_df['observation_date'] = pd.to_datetime(merged_df['observation_date'])

# Deliquency interpolate between points 
merged_df['delinquency_rate_interpolated'] = merged_df['delinquency_rate'].interpolate(method='linear')

# Add year column
merged_df['year'] = merged_df['observation_date'].dt.year

# Calculate Z-scores for MBS and deliquency where they are both present
mask = merged_df['mbs_price'].notna() & merged_df['delinquency_rate_interpolated'].notna()
merged_df.loc[mask, 'mbs_z_score'] = (
    (merged_df.loc[mask, 'mbs_price'] - merged_df.loc[mask, 'mbs_price'].mean()) / 
    merged_df.loc[mask, 'mbs_price'].std()
)
merged_df.loc[mask, 'delinq_z_score'] = (
    (merged_df.loc[mask, 'delinquency_rate_interpolated'] - 
     merged_df.loc[mask, 'delinquency_rate_interpolated'].mean()) / 
    merged_df.loc[mask, 'delinquency_rate_interpolated'].std()
)

# Export to CSV
merged_df.to_csv('merged_mortgage_data.csv', index=False)

